# CAR-33 Lightcurve Fitting

In [ ]:
import batman
import numpy as np
import matplotlib.pyplot as plt
from tshirt.pipeline import spec_pipeline
import os
from copy import deepcopy
import pdb
from astropy.io import fits, ascii
from scipy.optimize import curve_fit
%matplotlib inline

<div class='alert alert-info'>
    <font size='3'><b>These cells contain instructions. Edit the paths below.</b></font>
</div>

In [ ]:
par_path = '.'
#par_file = '../pipe_process/default_nc_grism_params.yaml'
par_file = '../pipe_process/simple_nc_grism_params.yaml'

In [ ]:
params = batman.TransitParams()
params.t0 = 10193.58 / (3600. * 24.) ## transit center, seconds. Using orbital phase start window
params.per = 399830.1696 / (3600. * 24.) ## seconds, period in APT
params.rp = 0.0805  ## rp/r*, Torres. et al. 2010
params.a = 8.87 ## a/r*, Torres et al. 2010
params.inc = 83.5 ## inclination, Triaud et al. 2015
params.ecc = 0.1074  ## eccentricity, Bonomo et al. 2017
params.w = 94.4 ## longitude of periastron, Bonomo et al. 2017
params.limb_dark =  "nonlinear" ## type of limb darkening model
params.u = [0.17,0.473,-0.586,0.215] 

In [ ]:
p2 = deepcopy(params)
def model_to_optimize(x,oot,rp):
    p2.rp = rp
    m = batman.TransitModel(p2, x)    #initializes model
    flux = m.light_curve(p2)          #calculates light curve
    return flux * oot

In [ ]:
def fit_time_series(par_path,par_file,pipeType='phot',
                    returnResults=False,nbins=1,bin_ind=0):
    
    full_param_path = os.path.join(par_path,par_file)
    
    if pipeType == 'phot':
        
        phot = phot_pipeline.phot(full_param_path)
        t1, t2 = phot.get_tSeries()
        t1.colnames, t2.colnames

        orig_header = fits.getheader(phot.photFile,extname='ORIG HEADER')
        timeKey = 'Time (JD)'
    else:
        spec = spec_pipeline.spec(full_param_path)
        t1, t2 = spec.get_wavebin_series(nbins=nbins)
        timeKey = 'Time'

    
    trel = t1[timeKey] - np.min(t1[timeKey]) + (+13.3/(3600. * 24.))
    pts_norm = trel < 0.05
    tab_ind = bin_ind + 1 ## first table column is time, so add 1
    norm_val = np.mean(t1[t1.colnames[tab_ind]][pts_norm])
    norm_y = t1[t1.colnames[tab_ind]] / norm_val
    err_y = t2[t2.colnames[tab_ind]] / norm_val
    
    ## Calculate model at same times
    m_at_ints = batman.TransitModel(deepcopy(params), trel)    #initializes model
    m_flux_at_ints = m_at_ints.light_curve(params)          #calculates light curve
    
    resid = (norm_y - m_flux_at_ints)
    
    good_pt = np.abs(resid) < 0.01
    
    ## fit a new model
    p0 = [1.0, 0.1]
    popt, pcov = curve_fit(model_to_optimize,xdata=trel[good_pt],ydata=norm_y[good_pt],
                           p0=p0,sigma=err_y[good_pt])
    
    bestFit = model_to_optimize(trel,*popt)
    
    
#     print("Rp = {} +/- {}".format(popt[1],np.sqrt(pcov[1])))
#     print("Orig Rp = {}".format(params.rp))
    depth = popt[1]**2 * 1000.
    depth_err = np.sqrt(pcov[1,1]) * 2. * popt[1] * 1000.
    print("depth = {} +/- {} ppt".format(depth,depth_err))
    print("Expected depth = {} ppt".format(params.rp**2 * 1000.))
    
    fig, ax = plt.subplots()
    plt.plot(trel,norm_y,'.',label='data')
    plt.plot(trel,m_flux_at_ints,label='orig')
    plt.plot(trel,bestFit,label='best fit')
    plt.legend()
    fig, ax = plt.subplots()
    
    
    plt.plot(trel[good_pt],resid[good_pt] * 1e6)
    plt.ylabel("Diff (ppm)")
    
    if returnResults == True:
        resultDict = {}
        resultDict['trel'] = trel
        resultDict['bestFit'] = bestFit
        resultDict['popt'] = popt
        resultDict['pcov'] = pcov
        resultDict['depth'] = depth
        resultDict['depth err'] = depth_err
        resultDict['wavename'] = t1.colnames[tab_ind]
        return resultDict

## Check the Broadband Depth

In [ ]:
bb_res = fit_time_series(par_path,par_file,pipeType='spec',returnResults=True)

## Check the Binned Spectrum

<div class='alert alert-info'>
    <font size='3'><b>Choose a number of bins.</b></font>
</div>

In [ ]:
nbins=10

In [ ]:
binned_wave = []
binned_depth, binned_depth_err = [], []
for bin_ind in np.arange(nbins):
    res = fit_time_series(par_path,par_file,
                          pipeType='spec',nbins=nbins,bin_ind=bin_ind,
                          returnResults=True)
    binned_wave.append(float(res['wavename'].split('um')[0]))
    binned_depth.append(res['depth'])
    binned_depth_err.append(res['depth err'])

In [ ]:
med_err = np.median(binned_depth_err)
noise_floor_lim = 0.1 ## 100 ppm criterion
floor_comb = np.sqrt(med_err**2 + noise_floor_lim**2)

In [ ]:
exp_depth = 0.0805**2

fig, ax = plt.subplots()
ax.errorbar(binned_wave,binned_depth,yerr=binned_depth_err,fmt='o')
ax.axhline(exp_depth * 1e3,color='red')
ax.axhline(exp_depth * 1e3 - floor_comb,linestyle='dashed',color='red')
ax.axhline(exp_depth * 1e3 + floor_comb,linestyle='dashed',color='red')
ax.set_xlabel("Wavelength ($\mu$m)")
ax.set_ylabel("Depth (ppt)")



## Save the Residuals for Allan Variance Analysis

In [ ]:
full_param_path = os.path.join(par_path,par_file)
spec = spec_pipeline.spec(full_param_path)
HDUList_dyn = fits.open(spec.dyn_specFile())
dynSpec = HDUList_dyn["DYNAMIC SPEC"].data
errDynHDU = HDUList_dyn["DYN SPEC ERR"].data
tdyn = HDUList_dyn['TIME'].data

In [ ]:
dispSt, dispEnd = spec.param['dispPixels']
nPix = dispEnd - dispSt
model2D = np.tile(bb_res['bestFit'],[nPix,1]).T

In [ ]:
resid2D = dynSpec[:,dispSt:dispEnd] - model2D

In [ ]:
plt.imshow(dynSpec[:,dispSt:dispEnd],vmin=0.994,vmax=1.003)


In [ ]:
plt.imshow(resid2D,vmin=-0.002,vmax=0.002)

In [ ]:
primHDU = fits.PrimaryHDU(resid2D)
primHDU.name = "RESID"
timeHDU = fits.ImageHDU(tdyn)
timeHDU.name = "TIME"
residErrHDU = fits.ImageHDU(errDynHDU[:,dispSt:dispEnd])
residErrHDU.name = "RESID ERR"
outHDUList = fits.HDUList([primHDU,residErrHDU,timeHDU])
outHDUList.writeto('../allan_variance/resid2d.fits',overwrite=True)